In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import nltk
import re
from nltk.corpus import stopwords

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
Train_data = pd.read_csv('/kaggle/input/threads-an-instagram-app-reviews/threads_reviews.csv')

In [3]:
Train_data

,source,review_description,rating,review_date
0,Google Play,Meh. Not the greatest experience on a Chromebo...,2,2023-07-08 14:18:24
1,Google Play,Pretty good for a first launch!! Its easy to u...,3,2023-07-19 20:52:48
2,Google Play,"For a brand new app, it's very well optimized....",3,2023-07-06 23:03:11
3,Google Play,"Great app with a lot of potential! However, th...",3,2023-07-10 00:53:25
4,Google Play,"The app is good, but it needs a lot of functio...",3,2023-07-06 16:57:43
...,...,...,...,...
32905,App Store,This killed my dog. Mark zuckerburg strangled ...,1,2023-07-06 01:23:55
32906,App Store,Add Search and hashtag like Twitter !,1,2023-07-19 08:01:06
32907,App Store,bad twister,1,2023-07-17 06:39:13
32908,App Store,Yet another trash from Meta.,1,2023-07-07 17:47:16


In [4]:
Train_data['rating'] = Train_data['rating'].apply(lambda x:x-1)
Train_data['rating'].value_counts()

4    15559
0     9726
3     3244
2     2585
1     1796
Name: rating, dtype: int64

In [6]:
trian_text_data = Train_data['review_description'].values
train_text_data = trian_text_data.tolist()

In [7]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
s_words = set(stopwords.words("english"))
train_text_data_final = []
for review in trian_text_data : 
    review = re.sub("[^a-zA-Z]", ' ', review)
    review = review.lower().split()
    review = [x for x in review if x not in s_words]
    review = " ".join(review)
    train_text_data_final.append(review)

trian_text_data = train_text_data_final

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
tokenizer = Tokenizer(num_words = 32910 , oov_token = "<OOV>" )
tokenizer.fit_on_texts(train_text_data)

In [10]:
word_index = tokenizer.index_word

In [11]:
sequance = tokenizer.texts_to_sequences(train_text_data)

In [12]:
paded = pad_sequences(sequance)

In [13]:
labels = Train_data['rating'].values
labels = labels.tolist()

In [14]:
len(labels)

32910

In [15]:
train_data = paded[:22400]
train_label = labels[:22400]

train_data = np.array(train_data)
train_label = np.array(train_label)

train_label = np.expand_dims(train_label , axis= 1 )

test_data = paded[22400:]
test_label = labels[22400:]

test_data = np.array(test_data)


test_label = np.array(test_label)
test_label = np.expand_dims(test_label , axis= 1 )

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(32910 ,16 , input_length=1000) , 
    tf.keras.layers.GRU(128, return_sequences=True) ,
    tf.keras.layers.SimpleRNN(64) ,
    tf.keras.layers.Dense(1000 , activation='relu') ,
      tf.keras.layers.Dropout(0,2) ,
    tf.keras.layers.Dense(300 , activation='relu'),
    tf.keras.layers.Dense(5 , activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 16)          526560    
                                                                 
 gru (GRU)                   (None, 1000, 128)         56064     
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                12352     
                                                                 
 dense (Dense)               (None, 1000)              65000     
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 300)               300300    
                                                                 
 dense_2 (Dense)             (None, 5)                 1

In [15]:
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.001) , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics= ['accuracy'])

In [ ]:
model.fit(x= train_data ,y=train_label ,batch_size=32 ,epochs= 30)

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


700/700 [==============================] - 573s 807ms/step - loss: 1.0819 - accuracy: 0.6081
Epoch 2/30
700/700 [==============================] - 537s 767ms/step - loss: 0.8644 - accuracy: 0.6975
Epoch 3/30
700/700 [==============================] - 537s 767ms/step - loss: 0.7601 - accuracy: 0.7313
Epoch 4/30
700/700 [==============================] - 535s 764ms/step - loss: 1.0245 - accuracy: 0.6082
Epoch 5/30
700/700 [==============================] - 534s 763ms/step - loss: 1.0878 - accuracy: 0.6118
Epoch 6/30
700/700 [==============================] - 534s 763ms/step - loss: 0.7994 - accuracy: 0.7119
Epoch 7/30
700/700 [==============================] - 533s 761ms/step - loss: 0.7272 - accuracy: 0.7334
Epoch 8/30
 10/700 [..............................] - ETA: 8:51 - loss: 0.7090 - accuracy: 0.7563

In [ ]:
accuracy , loss = model.evaluate(x = test_data , y = test_label , batch_size = 32 )